The main program that uses the database and defined python scripts to run the analysis

In [3]:
#This non-descript folder contains all the standalone python modules
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

In [4]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [ ]:
scores = []
scores = list(map(runSimulation, range(80), repeat(seasonsData), repeat(qualiResultsData), repeat(driversData), 
                    repeat(constructorsData), repeat(enginesData)))
print(np.median(scores))

So, now when the data has been analysed, it's time to use it to predict the next qualification's results! In this simplified version, the grand prix is specified by the parameter circuit.

In [6]:
(drivers, constructors, engines) = runSimReturnDrivConstEng(seasonsData, qualiResultsData, 
                                                            driversData, constructorsData, enginesData)

In [7]:
newDrivers = json.load(open('newDrivers.json'))["drivers"]
#print(newDrivers)

driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = drivers[int(did)].constructor.name
with open('driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

In [8]:
circuit = 7

driverResults = {} # {did: {position: amount}}
for did, cid in newDrivers.items():
    driverResults[int(did)] = {}
for i in range(1000):
    scoreList = predictQualiResults(circuit, newDrivers, drivers, constructors, engines)
    for i, drivRes in enumerate(scoreList):
        if i not in driverResults[drivRes[0]]:
            driverResults[drivRes[0]][i] = 0
        driverResults[drivRes[0]][i] += 1
        
for did, res in driverResults.items():
    print("{0} ({1}):".format(drivers[int(did)].name, drivers[int(did)].constructor.name))
    for pos, amount in sorted(res.items(), key=lambda posAmount: posAmount[0]):
        print("\t{0}: {1} %".format(pos + 1, amount / 10.0))
        
with open('predictions.json', 'w') as fp:
    json.dump(driverResults, fp)

#for i, res in enumerate(scoreList):
    #print("{0}: {1} ({2})".format(i+1, drivers[res[0]].name, drivers[res[0]].constructor.name))        

Lewis Hamilton (Mercedes):
	1: 10.8 %
	2: 24.2 %
	3: 29.6 %
	4: 15.8 %
	5: 10.5 %
	6: 6.1 %
	7: 0.2 %
	8: 0.2 %
	18: 0.2 %
	19: 1.1 %
	20: 1.3 %
Fernando Alonso (McLaren):
	7: 2.1 %
	8: 7.7 %
	9: 15.3 %
	10: 20.3 %
	11: 16.9 %
	12: 14.1 %
	13: 10.6 %
	14: 6.3 %
	15: 2.8 %
	16: 1.5 %
	17: 0.3 %
	18: 0.2 %
	19: 0.5 %
	20: 1.4 %
Kimi Räikkönen (Ferrari):
	1: 32.3 %
	2: 33.2 %
	3: 14.6 %
	4: 8.5 %
	5: 5.7 %
	6: 3.2 %
	7: 0.2 %
	19: 0.9 %
	20: 1.4 %
Sebastian Vettel (Ferrari):
	1: 53.5 %
	2: 23.4 %
	3: 10.7 %
	4: 4.7 %
	5: 3.2 %
	6: 1.6 %
	7: 0.1 %
	18: 0.2 %
	19: 0.6 %
	20: 2.0 %
Romain Grosjean (Haas F1 Team):
	7: 0.4 %
	8: 0.8 %
	9: 4.8 %
	10: 10.4 %
	11: 19.8 %
	12: 20.8 %
	13: 18.5 %
	14: 12.9 %
	15: 5.2 %
	16: 2.0 %
	17: 1.2 %
	18: 0.2 %
	19: 0.6 %
	20: 2.4 %
Nico Hülkenberg (Renault):
	4: 0.1 %
	5: 1.0 %
	6: 11.8 %
	7: 47.1 %
	8: 13.6 %
	9: 11.2 %
	10: 6.1 %
	11: 2.4 %
	12: 2.3 %
	13: 0.9 %
	14: 0.8 %
	16: 0.1 %
	18: 0.2 %
	19: 0.7 %
	20: 1.7 %
Sergio Pérez (Force India):
	5: 0.3 %
	